# ⚙️ **CLEANED DATA IMPORT**

In [50]:
import pandas as pd
import numpy as np
import os

DATA_PATH = '../data/cleaned'

POI_FILENAME = 'poi_df_cleaned.csv'
SITE_FILENAME = 'site_df_cleaned.csv'
SALARY_FILENAME = 'salary_df_cleaned.csv'
GEOREF_FILENAME = 'georef_df_cleaned.csv'
STOCK_FILENAME = 'stock_df_cleaned.csv'
SALES_FILENAME = 'sales_df_cleaned.csv'
POPULATION_FILENAME = 'population_df_cleaned.csv'
POVERTY_FILENAME = 'poverty_df_cleaned.csv'
REAL_ESTATE_FILENAME = 'real_estate_df_cleaned.csv'

poi_df = pd.read_csv(os.path.join(DATA_PATH, POI_FILENAME))
site_df = pd.read_csv(os.path.join(DATA_PATH, SITE_FILENAME))
salary_df = pd.read_csv(os.path.join(DATA_PATH, SALARY_FILENAME))
georef_df = pd.read_csv(os.path.join(DATA_PATH, GEOREF_FILENAME))
stock_df = pd.read_csv(os.path.join(DATA_PATH, STOCK_FILENAME))
sales_df = pd.read_csv(os.path.join(DATA_PATH, SALES_FILENAME))
population_df = pd.read_csv(os.path.join(DATA_PATH, POPULATION_FILENAME))
poverty_df = pd.read_csv(os.path.join(DATA_PATH, POVERTY_FILENAME))
real_estate_df = pd.read_csv(os.path.join(DATA_PATH, REAL_ESTATE_FILENAME))

### CLEANING

##### DF_SALES CLEANING

In [3]:
# SALES_DF: Suppression des doublons > nous passons de 4,3M de lignes à 3,821M
sales_df = sales_df.drop_duplicates()
sales_df.shape

(3448398, 13)

In [4]:
# SALES_DF: Check si les doublons on été enlevés : OK
sales_df.duplicated().sum()

np.int64(0)

In [5]:
# SALES_DF: Suppression des prix au m2 supérieur à 30K€ et inférieur à 1K€ > nous passons à 3,3399M de lignes
sales_df = sales_df[(sales_df['sales_price_m2'] <= 30000) & (sales_df['sales_price_m2'] >= 1000)]
sales_df.shape

(3448398, 13)

In [6]:
# SALES_DF:
s2 = (sales_df['sales_amount']
             .value_counts()
             .loc[sales_df['sales_amount'].value_counts() > 10])

In [7]:
# SALES_DF:
sales_df = sales_df[sales_df['sales_amount'] > 1] # on enlève les 166 fois ou sales_amount = 1€
sales_df.shape

(3448398, 13)

In [8]:
# SALES_DF: changement du type sales_date en datetime
sales_df['sales_date'] = pd.to_datetime(sales_df['sales_date'])
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3448398 entries, 0 to 3448397
Data columns (total 13 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   sales_date                 datetime64[ns]
 1   sales_amount               float64       
 2   street_number              float64       
 3   street_code                object        
 4   street_name                object        
 5   nom_commune                object        
 6   municipality_code          object        
 7   premise_type               object        
 8   surface                    float64       
 9   number_of_principal_rooms  int64         
 10  sales_price_m2             float64       
 11  latitude                   float64       
 12  longitude                  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(5)
memory usage: 342.0+ MB


##### DF_SALARY CLEANING

In [9]:
# DF_SALARY: ROUND avg_net_salary
salary_df['avg_net_salary'] = salary_df['avg_net_salary'].round()
salary_df.head()

,municipality_code,avg_net_salary,year
0,01004,26471.0,2019
1,01007,26341.0,2019
2,01014,25897.0,2019
3,01024,25695.0,2019
4,01025,26055.0,2019


##### DF_REAL_ESTATE CLEANING

In [10]:
# DF_REAL_ESTATE: suppression des nulls
real_estate_df = real_estate_df.dropna(axis=1)
real_estate_df.isnull().sum()

municipality_code         0
intensite_tension_immo    0
rental_max_apartment      0
rental_min_apartment      0
rental_med_all            0
rental_max_all            0
rental_min_all            0
dtype: int64

##### DF_SITE CLEANING

In [11]:
# SITE_DF: tri avec les données entre parenthèses de la colonne "name" inclues

import re

site_df['data_inside_parenthesis'] = site_df['name'].apply(lambda x: re.search(r'\((.*?)\)', x).group(1) if re.search(r'\((.*?)\)', x) else '')
site_df

#suppression de la colonne "name" dans un second temps

site_df.drop(columns=["name"])

#check pour savoir les informations présentes dans la colonne "poi", et si elles correspondent aux valeurs présentes dans la colonne "type"
print (site_df["poi"].value_counts())
print (site_df["data_inside_parenthesis"].value_counts().head(50))

#faire un mapping des colonnes poi, qui sont en fait plus pertinentes que celles de la colonne "type"

poi
castle            5611
2                 5207
water             4500
museum            3400
theatre           2439
forest            2233
beach             1010
protected_area     927
wetland            784
cinema             667
vineyard           641
golf_course        612
theme_park         358
cliff              337
marina             255
water_park         239
1                  236
zoo                234
valley             223
ridge              213
islet              163
casino             158
volcano            145
park               132
meadow              78
monument            76
sand                35
heritage            32
viewpoint           21
cave_entrance       15
wreck               12
rock                11
waterfall           10
attraction           8
dune                 5
national_park        4
allotments           2
geyser               1
Name: count, dtype: int64
data_inside_parenthesis
                       9899
Monument historique    5090
Eau             

In [12]:
# SITE_DF: création d'un dictionnaire intégrant toutes les différentes valeurs inclues dans la colonne "poi"
s = site_df["poi"].value_counts()[site_df["poi"]]
{k: "toto" for k in s.index}

{'1': 'toto',
 '2': 'toto',
 'zoo': 'toto',
 'dune': 'toto',
 'park': 'toto',
 'rock': 'toto',
 'sand': 'toto',
 'beach': 'toto',
 'cliff': 'toto',
 'islet': 'toto',
 'ridge': 'toto',
 'water': 'toto',
 'wreck': 'toto',
 'casino': 'toto',
 'castle': 'toto',
 'cinema': 'toto',
 'forest': 'toto',
 'geyser': 'toto',
 'marina': 'toto',
 'meadow': 'toto',
 'museum': 'toto',
 'valley': 'toto',
 'theatre': 'toto',
 'volcano': 'toto',
 'wetland': 'toto',
 'heritage': 'toto',
 'monument': 'toto',
 'vineyard': 'toto',
 'viewpoint': 'toto',
 'waterfall': 'toto',
 'allotments': 'toto',
 'attraction': 'toto',
 'theme_park': 'toto',
 'water_park': 'toto',
 'golf_course': 'toto',
 'cave_entrance': 'toto',
 'national_park': 'toto',
 'protected_area': 'toto'}

In [13]:
# SITE_DF: création d'un dictionnaire avec les catégories associées aux valeurs de la colonne POI

category_dict = {'1': 'Patrimoine',
 '2': 'Patrimoine',
 'zoo': 'Entertainment',
 'dune': 'Nature',
 'park': 'Nature',
 'rock': 'Nature',
 'sand': 'Nature',
 'beach': 'Nature',
 'cliff': 'Nature',
 'islet': 'Nature',
 'ridge': 'Nature',
 'water': 'Nature',
 'wreck': 'Patrimoine',
 'casino': 'Entertainment',
 'castle': 'Patrimoine',
 'cinema': 'Culture',
 'forest': 'Nature',
 'geyser': 'Nature',
 'marina': 'Nature',
 'meadow': 'Nature',
 'museum': 'Culture',
 'valley': 'Nature',
 'theatre': 'Culture',
 'volcano': 'Nature',
 'wetland': 'Nature',
 'heritage': 'Patrimoine',
 'monument': 'Patrimoine',
 'vineyard': 'Nature',
 'viewpoint': 'Nature',
 'waterfall': 'Nature',
 'allotments': 'Patrimoine',
 'attraction': 'Entertainment',
 'theme_park': 'Entertainment',
 'water_park': 'Entertainment',
 'golf_course': 'Entertainment',
 'cave_entrance': 'Culture',
 'national_park': 'Nature',
 'protected_area': 'Nature'}

In [14]:
# SITE_DF: création de la colonne "catégorie"
site_df["Category"] = site_df["poi"].map(category_dict)
site_df

,poi,name,latitude,longitude,municipality_code,importance,name_reprocessed,data_inside_parenthesis,Category
0,1,Fortifications de Vauban (Patrimoine mondial),46.396160,1.472600,36134,0.139527,Fortifications de Vauban,Patrimoine mondial,Patrimoine
1,1,Chemins de Saint-Jacques-de-Compostelle en Fra...,46.313695,1.478772,23258,0.137821,Chemins de Saint-Jacques-de-Compostelle en Fra...,Patrimoine mondial,Patrimoine
2,1,Canal du Midi (Patrimoine mondial),43.332709,3.219890,34032,0.129531,Canal du Midi,Patrimoine mondial,Patrimoine
3,1,Bassin minier du Nord-Pas de Calais (Patrimoin...,50.455895,2.965034,62427,0.127170,Bassin minier du Nord-Pas de Calais,Patrimoine mondial,Patrimoine
4,1,"Les Causses et les Cévennes, paysage culturel ...",44.346210,3.613406,48050,0.124981,"Les Causses et les Cévennes, paysage culturel ...",Patrimoine mondial,Patrimoine
...,...,...,...,...,...,...,...,...,...
31029,viewpoint,Tour Eiffel 3e étage (Point de vue),48.858262,2.294497,75056,0.053782,Tour Eiffel 3e étage,Point de vue,Nature
31030,theme_park,Jardin d'Acclimatation (Parc à thème),48.878145,2.264686,75056,0.087097,Jardin d'Acclimatation,Parc à thème,Entertainment
31031,theme_park,Foire du Trône (Parc à thème),48.832003,2.404337,75056,0.060000,Foire du Trône,Parc à thème,Entertainment
31032,golf_course,Golf du Bois de Boulogne (Terrain de golf),48.857776,2.232651,75056,0.060073,Golf du Bois de Boulogne,Terrain de golf,Entertainment


In [13]:
poi_df.info()
site_df.info()
salary_df.info()
georef_df.info() 
stock_df.info() 
sales_df.info()
population_df.info() 
poverty_df.info()
real_estate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   poi                26216 non-null  object 
 1   latitude           26216 non-null  float64
 2   longitude          26216 non-null  float64
 3   municipality_code  26216 non-null  object 
 4   importance         26216 non-null  float64
 5   name_reprocessed   26216 non-null  object 
dtypes: float64(3), object(3)
memory usage: 1.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31034 entries, 0 to 31033
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   poi                      31034 non-null  object 
 1   name                     31034 non-null  object 
 2   latitude                 31034 non-null  float64
 3   longitude                31034 non-null  float64
 4   municipality_code  

In [ ]:
poi_df.head(1)

In [ ]:
site_df.head(1) 

In [ ]:
salary_df.head(1)

In [ ]:
georef_df.head(1) 

In [ ]:
stock_df.head(1) 

In [ ]:
sales_df.head(1)

In [ ]:
population_df.head(1) 

In [ ]:
poverty_df.head(1)

In [ ]:
real_estate_df.head(1)

# 🧪 **DATA TRANSFORMATION**

### KPIS AGGREGATION BY DEPARTMENT

##### 1. TOURISM

In [55]:
# 1.1 Nombre de sites touristiques par département
# Joindre les informations de géolocalisation pour obtenir les départements
site_dep_df = site_df.merge(georef_df[['municipality_code', 'department_code']], on='municipality_code')

# Calculer le nombre de sites touristiques par département
num_sites_per_department = site_dep_df.groupby('department_code')['poi'].count().reset_index()
num_sites_per_department.rename(columns={'poi': 'num_sites'}, inplace=True)
num_sites_per_department

,department_code,num_sites
0,01,508
1,02,217
2,03,323
3,04,224
4,05,313
...,...,...
91,91,325
92,92,152
93,93,90
94,94,101


In [66]:
# 1.2 Importance moyenne des sites par département
# Calculer l'importance moyenne des sites touristiques par département
avg_site_importance_per_department = site_dep_df.groupby('department_code')['importance'].mean().reset_index() 
avg_site_importance_per_department.rename(columns={'importance': 'avg_site_importance'}, inplace=True)
avg_site_importance_per_department

,department_code,avg_site_importance
0,01,0.067051
1,02,0.072294
2,03,0.066258
3,04,0.065725
4,05,0.069074
...,...,...
91,91,0.066098
92,92,0.067027
93,93,0.063537
94,94,0.066328


In [68]:
# 1.3 Stock de logement par département
# Joindre les informations de géolocalisation pour obtenir les départements
stock_dep_df = stock_df.merge(georef_df[['municipality_code', 'department_code']], on='municipality_code')

# Calculer le stock de logement par département (nombre total de logements)
total_stock_per_department = stock_dep_df.groupby('department_code')['nb_tot_housing'].sum().reset_index()
total_stock_per_department.rename(columns={'nb_tot_housing': 'total_stock'}, inplace=True)
total_stock_per_department

,department_code,total_stock
0,01,1781912
1,02,1829844
2,03,1462151
3,04,755894
4,05,763007
...,...,...
95,95,3134354
96,971,1203232
97,972,1161753
98,973,384652


##### 2. REAL ESTATE

In [72]:
# 2.1 Rentabilité locative au m² par département
# Joindre les informations de géolocalisation pour obtenir les départements
real_estate_dep_df = real_estate_df.merge(georef_df[['municipality_code', 'department_code', 'department_name']], on='municipality_code')

# Calculer la rentabilité locative moyenne au m² par département
real_estate_dep_df['avg_rental_yield'] = (real_estate_dep_df['rental_max_all'] + real_estate_dep_df['rental_min_all']) / 2
rental_yield_per_department = real_estate_dep_df.groupby(['department_code', 'department_name'])['avg_rental_yield'].mean().reset_index()
rental_yield_per_department.rename(columns={'avg_rental_yield': 'avg_rental_yield'}, inplace=True)
rental_yield_per_department

,department_code,department_name,avg_rental_yield
0,01,Ain,12.253130
1,02,Aisne,9.627683
2,03,Allier,9.244937
3,04,Alpes-de-Haute-Provence,11.311378
4,05,Hautes-Alpes,11.714241
...,...,...,...
91,91,Essonne,16.469145
92,92,Hauts-de-Seine,25.354306
93,93,Seine-Saint-Denis,21.065125
94,94,Val-de-Marne,22.158404


In [73]:
# 2.2 Tension immobilière par département
# Calculer la tension immobilière par département
housing_tension_per_department = real_estate_dep_df.groupby('department_code')['intensite_tension_immo'].mean().reset_index()
housing_tension_per_department.rename(columns={'intensite_tension_immo': 'avg_housing_tension'}, inplace=True)

In [75]:
# 2.3 Part de maisons secondaires par département
# Calculer la part de maisons secondaires par département
secondary_home_rate_per_department = stock_dep_df.groupby('department_code')['secondary_home_rate'].mean().reset_index()
secondary_home_rate_per_department.rename(columns={'secondary_home_rate': 'avg_secondary_home_rate'}, inplace=True)
secondary_home_rate_per_department

,department_code,avg_secondary_home_rate
0,01,0.138884
1,02,0.102122
2,03,0.132708
3,04,0.397630
4,05,0.400697
...,...,...
95,95,0.071613
96,971,0.076645
97,972,0.068388
98,973,0.062976


In [76]:
# 2.4 Évolution du prix au m² par département
# Joindre les informations de géolocalisation pour obtenir les départements
sales_dep_df = sales_df.merge(georef_df[['municipality_code', 'department_code', 'department_name']], on='municipality_code')

# Calculer l'évolution du prix au m² par département
price_evolution = sales_dep_df.groupby(['department_code', 'department_name', 'sales_date'])['sales_price_m2'].mean().unstack().reset_index()
price_evolution['price_evolution'] = (price_evolution[price_evolution.columns[-1]] - price_evolution[price_evolution.columns[-2]]) / price_evolution[price_evolution.columns[-2]] * 100
price_evolution = price_evolution[['department_code', 'department_name', 'price_evolution']]
price_evolution.head()


sales_date,department_code,department_name,price_evolution
0,01,Ain,-26.355387
1,02,Aisne,-7.993162
2,03,Allier,-12.084001
3,04,Alpes-de-Haute-Provence,NaN
4,05,Hautes-Alpes,19.868180


In [77]:
# Calculer le prix moyen au m² des ventes immobilières par département
avg_price_per_m2_per_department = sales_dep_df.groupby(['department_code', 'department_name'])['sales_price_m2'].mean().reset_index()
avg_price_per_m2_per_department.rename(columns={'sales_price_m2': 'avg_sales_price_m2'}, inplace=True)
avg_price_per_m2_per_department.head()

,department_code,department_name,avg_sales_price_m2
0,01,Ain,2666.678222
1,02,Aisne,1622.058993
2,03,Allier,1623.745514
3,04,Alpes-de-Haute-Provence,2314.257869
4,05,Hautes-Alpes,2625.535305


##### 3. SECONDARY HOME

In [40]:
# 3.1 Superficie moyenne des logements vendus par départements

# Joindre les informations de géolocalisation pour obtenir les départements
real_estate_sales_dep = sales_df.merge(
    georef_df[['municipality_code', 'department_code', 'department_name']],
    on='municipality_code'
)

# Calculer la surface moyenne des logements vendus par municipalité
average_surface_municipality = real_estate_sales_dep.groupby('municipality_code')['surface'].mean().reset_index()
average_surface_municipality.rename(columns={'surface': 'average_surface'}, inplace=True)

# Joindre les informations de géolocalisation pour obtenir les départements
average_surface_municipality = average_surface_municipality.merge(
    georef_df[['municipality_code', 'department_code', 'department_name']],
    on='municipality_code'
)

# Calculer la surface moyenne des logements vendus par département
average_surface_department = average_surface_municipality.groupby(['department_code', 'department_name'])['average_surface'].mean().reset_index()
average_surface_department.rename(columns={'average_surface': 'avg_home_surface'}, inplace=True)

average_surface_department.head()

,department_code,department_name,avg_home_surface
0,01,Ain,103.252328
1,02,Aisne,101.221864
2,03,Allier,95.459944
3,04,Alpes-de-Haute-Provence,84.560073
4,05,Hautes-Alpes,80.168163


In [16]:
# Filtre sur un departement en particulier (exemple paris 75)
print(average_surface_department[average_surface_department['department_code'] == '75'])

   department_code department_name  avg_house_surface
72              75           Paris           55.11202


In [31]:
# 3.2 Évolution du % des maisons secondaires par département

# Filtrer les données pour les années 2008 et 2018
housing_2008 = stock_df[stock_df['year'] == 2008]
housing_2018 = stock_df[stock_df['year'] == 2018]

# Renommer les colonnes pour les années spécifiques
housing_2008 = housing_2008[['municipality_code', 'secondary_home_rate']].rename(columns={'secondary_home_rate': 'secondary_home_rate_2008'})
housing_2018 = housing_2018[['municipality_code', 'secondary_home_rate']].rename(columns={'secondary_home_rate': 'secondary_home_rate_2018'})

# Joindre les données pour les années 2008 et 2018 sur le code de municipalité
secondary_home_rate_comparison = housing_2008.merge(housing_2018, on='municipality_code')

# Remplacer les valeurs 0 de 2008 pour éviter la division par zéro
secondary_home_rate_comparison = secondary_home_rate_comparison.replace({'secondary_home_rate_2008': {0: np.nan}})

# Calculer l'évolution du pourcentage de maisons secondaires par municipalité
secondary_home_rate_comparison['secondary_home_rate_evolution'] = (
    (secondary_home_rate_comparison['secondary_home_rate_2018'] - secondary_home_rate_comparison['secondary_home_rate_2008']) / 
    secondary_home_rate_comparison['secondary_home_rate_2008']
) * 100

# Remplacer les valeurs infinies et NaN par 0
secondary_home_rate_comparison = secondary_home_rate_comparison.replace({'secondary_home_rate_evolution': {np.inf: np.nan, -np.inf: np.nan}})
secondary_home_rate_comparison['secondary_home_rate_evolution'] = secondary_home_rate_comparison['secondary_home_rate_evolution'].fillna(0)

# Joindre les informations de géolocalisation pour obtenir les départements
secondary_home_rate_comparison = secondary_home_rate_comparison.merge(
    georef_df[['municipality_code', 'department_code', 'department_name']],
    on='municipality_code'
)

# Calculer l'évolution moyenne du pourcentage de maisons secondaires par département
secondary_home_rate_evolution_department = secondary_home_rate_comparison.groupby(['department_code', 'department_name'])['secondary_home_rate_evolution'].mean().reset_index()
secondary_home_rate_evolution_department.rename(columns={'secondary_home_rate_evolution': 'secondary_home_rate_evolution'}, inplace=True)

secondary_home_rate_evolution_department.head(10)

,department_code,department_name,secondary_home_rate_evolution
0,01,Ain,-8.064238
1,02,Aisne,-8.667438
2,03,Allier,-5.866098
3,04,Alpes-de-Haute-Provence,3.350143
4,05,Hautes-Alpes,-4.386948
5,06,Alpes-Maritimes,-1.166718
6,07,Ardèche,-7.774843
7,08,Ardennes,4.684543
8,09,Ariège,-0.522571
9,10,Aube,2.349340


##### 4. LIFE QUALITY

In [44]:
# 4.1 Professionnels de santé pour 100 000 habitants par départements en 2023
DATA_PATH = '../data/cleaned'
HEALTH_FILENAME = 'health_df_cleaned.csv'

health_df = pd.read_csv(os.path.join(DATA_PATH, HEALTH_FILENAME))
health_df.head()

,department_code,department_name,ensemble des médecins,ensemble des médecins.1,dont généralistes,dont spécialistes,chirurg. dentistes,pharm.
0,01,Ain,1 162,174,99,75,53,78
1,02,Aisne,1 107,211,99,111,45,91
2,03,Allier,917,275,134,141,51,111
3,04,Alpes-de-Haute-Provence,483,291,165,125,54,103
4,05,Hautes-Alpes,705,503,291,213,72,135


In [46]:
# 4.2 Taux de criminalité pour 1000 habitants par départements en 2020

CRIMINALITY_FILENAME = 'criminality_df_cleaned.csv'

criminality_df = pd.read_csv(os.path.join(DATA_PATH, CRIMINALITY_FILENAME))
criminality_df.head()

,department_name,criminality_per_1000
0,Paris,"98,43"
1,Seine-Saint-Denis,"77,76"
2,Bouches-du-Rhône,"63,21"
3,Rhône,"61,83"
4,Hauts-de-Seine,"59,88"


In [48]:
# 4.3 Nombre de jours de soleil par an par départements

SUNNY_FILENAME = 'sunny_df_cleaned.csv'

sunny_df = pd.read_csv(os.path.join(DATA_PATH, SUNNY_FILENAME))
sunny_df.head()

,department_name,sunny_days_per_year
0,Ain,182
1,Aisne,142
2,Allier,161
3,Alpes de Haute Provence,241
4,Hautes-Alpes,232


##### POPULATION

In [ ]:
# 1.1 Salaire moyen par département
# Joindre les informations de géolocalisation pour obtenir les départements
salary_dep_df = salary_df.merge(georef_df[['municipality_code', 'department_code', 'department_name']], on='municipality_code')

# Calculer le salaire moyen par département
avg_salary_per_department = salary_dep_df.groupby(['department_code', 'department_name'])['avg_net_salary'].mean().reset_index().round()
avg_salary_per_department.rename(columns={'avg_net_salary': 'avg_salary'}, inplace=True)
avg_salary_per_department.head()

In [ ]:
# 1.2 Évolution de la population par département
# Joindre les informations de géolocalisation pour obtenir les départements
population_dep_df = population_df.merge(georef_df[['municipality_code', 'department_code', 'department_name']], on='municipality_code')

# Calculer l'évolution de la population par département (différence entre les années)
pop_evolution = population_dep_df.groupby(['department_code', 'department_name', 'year'])['population'].sum().unstack().reset_index()
pop_evolution['evolution'] = (pop_evolution[pop_evolution.columns[-1]] - pop_evolution[pop_evolution.columns[-2]]) / pop_evolution[pop_evolution.columns[-2]] * 100
pop_evolution = pop_evolution[['department_code', 'department_name', 'evolution']]

pop_evolution.head()

In [ ]:
# 1.3 Taux de pauvreté par département
# Joindre les données de population
poverty_df = poverty_df.merge(population_df[['municipality_code', 'population']], on='municipality_code', suffixes=('_poverty', '_population'))

# Merge avec georef pour avoir le department_name
poverty_df = poverty_df.merge(georef_df[['municipality_code', 'department_name']], on='municipality_code', how='left')

# Groupe par department_name pour calculer le taux de pauvreté par département
poverty_by_department = poverty_df.groupby('department_name').agg({
    'population_poverty': 'sum',
    'population_population': 'sum'
}).reset_index()

# Calculer le poverty_rate pour chaque département
poverty_by_department['poverty_rate'] = (poverty_by_department['population_poverty'] / poverty_by_department['population']) * 100

# Afficher le résultat
poverty_by_department.head()

# calcul = poverty_rate = poverty_population / total_population * 100
# il faut ensuite le DF qui a poverty_rate à georef pour récupérer le department_name

In [ ]:
# 2.1 Nombre de sites touristiques par département
# Joindre les informations de géolocalisation pour obtenir les départements
site_dep_df = site_df.merge(georef_df[['municipality_code', 'department_code', 'department_name']], on='municipality_code')

# Calculer le nombre de sites touristiques par département
num_sites_per_department = site_dep_df.groupby(['department_code', 'department_name'])['poi'].count().reset_index()
num_sites_per_department.rename(columns={'poi': 'nb_sites'}, inplace=True)
num_sites_per_department.head()

In [ ]:
# 2.2 Importance moyenne des sites par département
# Calculer l'importance moyenne des sites touristiques par département
avg_site_importance_per_department = site_dep_df.groupby(['department_code', 'department_name'])['importance'].mean().reset_index()
avg_site_importance_per_department.rename(columns={'importance': 'avg_site_importance'}, inplace=True)
avg_site_importance_per_department.head()

In [ ]:
# 2.3 Stock de logement par département
# Joindre les informations de géolocalisation pour obtenir les départements
stock_dep_df = stock_df.merge(georef_df[['municipality_code', 'department_code', 'department_name']], on='municipality_code')

# Calculer le stock de logement par département (nombre total de logements)
total_stock_per_department = stock_dep_df.groupby(['department_code', 'department_name'])['nb_tot_housing'].sum().reset_index()
total_stock_per_department.rename(columns={'nb_tot_housing': 'total_stock'}, inplace=True)
total_stock_per_department.head()

# 🚀 ENRICHED EXPORT